In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers=[21,22,4132,-54,32,-86,34,43]

tf_dataset =tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
4132
-54
32
-86
34
43



Iterate through first n elements in tf dataset

In [4]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
4132


In [6]:
tf_dataset=tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
4132
32
34
43


In [7]:
tf_dataset=tf_dataset.map(lambda x: x*79)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1659
1738
326428
2528
2686
3397


Shuffe

In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1738
326428
2528
2686
1659
3397


Batching

In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1738 1659]
[2528 2686]
[  3397 326428]


Perform all of the above operations in one shot

In [10]:

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[  1584 297504]
[1512 2448]
[2304 3096]



Images

In [12]:
images_ds=tf.data.Dataset.list_files('images/*/*',shuffle=False)

In [18]:

image_count=len(images_ds)
print(image_count)

130


In [14]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\cat\\A cat appears to have caught the....jpg'


In [16]:
images_ds=images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\Stray Cat Alliance \xc2\xbb Building a No Kill....jpg'
b'images\\dog\\Genes contribute to dog breeds_ iconic....jpg'
b'images\\dog\\The Importance of Walking Your Dog....jpg'


In [17]:
class_names=["cat","dog"]

In [19]:
train_size=int(image_count*0.8)
train_ds=images_ds.take(train_size)
test_ds=images_ds.skip(train_size)

In [20]:
len(train_ds)

104

In [22]:
len(test_ds)

26

In [23]:
def get_label (file_path):
    import os
    parts=tf.strings.split(file_path,os.path.sep)
    return parts[-2]

In [25]:
get_label("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")


<tf.Tensor: shape=(), dtype=string, numpy=b'cat'>

In [26]:
def process_image(file_path):
    label=get_label(file_path)
    img=tf.io.read_file(file_path)
    img=tf.image.decode_jpeg(img)
    img=tf.image.resize(img, [128,128])
    return img , label

In [28]:
img,label=process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [29]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [30]:

for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[124.27344   85.35376   75.74048 ]
  [149.46606   88.408936  70.342285]
  [148.05078   87.05078   68.05078 ]
  ...
  [130.        75.        54.      ]
  [132.        77.        56.      ]
  [133.8125    78.8125    57.8125  ]]

 [[122.98169  107.32666  107.90088 ]
  [129.68701  101.046875  99.08594 ]
  [130.1128   103.33154  101.09326 ]
  ...
  [132.        77.        56.      ]
  [132.84766   77.84766   56.847656]
  [131.1875    76.1875    55.1875  ]]

 [[143.625    150.6477   150.8728  ]
  [143.58545  151.29468  151.14282 ]
  [145.33154  149.6836   150.52393 ]
  ...
  [129.        74.        53.      ]
  [132.85718   76.85718   53.857178]
  [130.375     74.375     51.375   ]]

 ...

 [[ 98.62402   99.62402   94.62402 ]
  [100.09277  101.09277   96.09277 ]
  [ 89.15283   90.15283   85.15283 ]
  ...
  [100.06738  100.06738   98.06738 ]
  [ 99.77051   99.77051   97.77051 ]
  [ 89.463135  89.463135  87.463135]]

 [[101.72046  102.72046   97.72046 ]
  [109.14258  110.142

In [31]:
def scale(image,label):
    return image/255 , label 

In [32]:
train_ds =train_ds.map(scale)

In [33]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.860769   0.79018074 0.6960631 ]
****Label:  b'dog'
****Image:  [0.6059436  0.5353554  0.45692402]
****Label:  b'dog'
****Image:  [0.01691177 0.0247549  0.01409314]
****Label:  b'cat'
****Image:  [0.46015984 0.4523167  0.29937553]
****Label:  b'dog'
****Image:  [0.5887083  0.6632181  0.41223767]
****Label:  b'dog'
